# 📥 Torrent → Google Drive Downloader (libtorrent 2.x, Colab-ready)

This notebook uses the modern libtorrent 2.x API (declarative settings via
a settings pack or constructor) and includes fallbacks to avoid runtime
errors on Google Colab. It installs a verified wheel (libtorrent==2.0.11)
when needed and uses a fail‑safe session creation pattern.

⚠️ Only download content you are legally allowed to download.

Tips:
- If Drive mount fails, the notebook will save files to `/content/torrents/`.
- This implementation focuses on the core torrent engine (no zipping
  or extra wrappers). It injects public trackers into magnet links by
  default to improve peer discovery in NAT-constrained Colab.

I validated the approach against libtorrent docs and PyPI (libtorrent 2.0.11
wheels exist for CPython 3.12+). ✅

In [ ]:
# Mount Google Drive (optional, robust fallback)
from google.colab import drive
import os

drive_mounted = False
try:
    if not os.path.exists('/content/drive/MyDrive'):
        drive.mount('/content/drive', force_remount=True)
    drive_mounted = os.path.exists('/content/drive/MyDrive')
    if drive_mounted:
        print('✅ Google Drive mounted: /content/drive/MyDrive')
except Exception as e:
    print('⚠️ Google Drive mount failed:', e)
    drive_mounted = False

if not drive_mounted:
    os.makedirs('/content/torrents', exist_ok=True)
    print('Using local path: /content/torrents')
else:
    # create a default folder inside MyDrive for convenience
    default_gdrive_path = '/content/drive/MyDrive/Torrent'
    os.makedirs(default_gdrive_path, exist_ok=True)
    print('Using Google Drive path:', default_gdrive_path)


In [ ]:
# Install libtorrent (2.0.11) and requests if missing
import sys
import subprocess
import importlib

def pip_install(pkg):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg])

try:
    import libtorrent as lt
    print('✅ libtorrent present:', getattr(lt, 'version', getattr(lt, '__version__', 'unknown')))
except Exception:
    print('⚠️ libtorrent not found. Installing libtorrent==2.0.11 ...')
    # libtorrent 2.0.11 has manylinux wheels for CPython 3.12+ on PyPI
    pip_install('libtorrent==2.0.11')
    importlib.invalidate_caches()
    import libtorrent as lt
    print('✅ libtorrent installed:', getattr(lt, 'version', getattr(lt, '__version__', 'unknown')))

try:
    import requests
except Exception:
    print('Installing requests...')
    pip_install('requests')
    importlib.invalidate_caches()
    import requests
    print('✅ requests installed')


In [ ]:
# Core modern, Colab-tuned torrent engine (settings constructor + settings_pack fallback)
import libtorrent as lt
import time
import os
import urllib.parse

def _create_session(settings: dict):
    """Try lt.session(settings) first; fallback to settings_pack + apply_settings.
    This handles differences across Python bindings/wheels.
    """
    try:
        ses = lt.session(settings)
        return ses
    except TypeError:
        sp = lt.settings_pack()
        for k, v in settings.items():
            # try to map a string key to the settings_pack enum
            try:
                key = getattr(lt.settings_pack, k)
            except Exception:
                # some bindings expose setting_by_name
                try:
                    key = lt.settings_pack.setting_by_name(k)
                except Exception:
                    continue
            if isinstance(v, bool):
                sp.set_bool(key, v)
            elif isinstance(v, int):
                sp.set_int(key, v)
            else:
                sp.set_str(key, str(v))
        ses = lt.session()
        ses.apply_settings(sp)
        return ses

def _add_trackers_to_magnet(magnet: str, trackers: list) -> str:
    if not magnet.startswith('magnet:'):
        return magnet
    for tr in trackers:
        magnet += '&tr=' + urllib.parse.quote(tr)
    return magnet

def download_colab(magnet_link: str, save_path: str,
                   max_peers: int = 400, peer_connect_timeout: int = 3,
                   add_trackers: bool = True):
    """Download a magnet to save_path using modern libtorrent 2.x patterns.
    Prints status every 5s. Exits when seeding.
    """
    # optional public trackers (helps tracker-less magnets)
    if add_trackers:
        trackers = [
            'udp://tracker.opentrackr.org:1337/announce',
            'udp://9.rarbg.to:2710/announce',
            'udp://tracker.torrent.eu.org:451/announce',
            'udp://exodus.desync.com:6969/announce',
            'udp://tracker.openbittorrent.com:6969/announce'
        ]
        magnet_link = _add_trackers_to_magnet(magnet_link, trackers)

    settings = {
        'user_agent': f'libtorrent/{getattr(lt, "version", "unknown")}',
        'listen_interfaces': '0.0.0.0:6881',
        'enable_dht': True,
        'enable_lsd': True,
        'announce_to_all_trackers': True,
        'announce_to_all_tiers': True,
        'connections_limit': max_peers,
        'max_peerlist_size': max_peers * 2,
        'peer_connect_timeout': peer_connect_timeout,
        'request_timeout': 10,
        'min_reconnect_time': 5,
        'active_downloads': -1,
        'active_seeds': -1,
        'allow_multiple_connections_per_ip': True,
    }
    ses = _create_session(settings)

    params = lt.parse_magnet_uri(magnet_link)
    params.save_path = save_path
    # sequential download flag set from the start
    params.flags |= lt.torrent_flags.sequential_download

    h = ses.add_torrent(params)
    print('🔗', magnet_link)

    # wait for metadata (magnet)
    while not h.status().has_metadata:
        print('📡 fetching metadata...')
        time.sleep(1)

    print('✅ metadata fetched — starting:', h.status().name)

    while not h.status().is_seeding:
        s = h.status()
        print(f"{s.progress*100:5.2f}% | ↓{s.download_rate/1000:6.1f} kB/s | "
              f"↑{s.upload_rate/1000:6.1f} kB/s | peers {s.num_peers}")
        time.sleep(5)

    print('🎉 COMPLETE — saved to', save_path)


In [ ]:
# --- Run: set your magnet and destination here, then run this cell ---
import os
from IPython.display import Markdown, display

# @markdown Paste magnet link below (or leave empty to test with Ubuntu ISO)
magnet_link = ""  # @param {type: 'string'}

# default save path chooses Drive if mounted, otherwise local storage
if 'drive_mounted' in globals() and drive_mounted:
    default_save = '/content/drive/MyDrive/Torrent'
else:
    default_save = '/content/torrents'

# @markdown Save path (folder will be created):
save_path = default_save  # @param {type: 'string'}
os.makedirs(save_path, exist_ok=True)

if not magnet_link:
    # test magnet (Ubuntu 24.04 ISO) — healthy swarm
    magnet_link = (
        'magnet:?xt=urn:btih:d33785346445580b435520974e4c93510e1329c3'
        '&dn=ubuntu-24.04.1-desktop-amd64.iso&tr=https://torrent.ubuntu.com/announce'
    )
    display(Markdown('**No magnet provided — using Ubuntu 24.04 test magnet (healthy swarm)**'))

# run the download (this blocks the cell until the torrent completes)
download_colab(magnet_link, save_path, max_peers=400, peer_connect_timeout=3, add_trackers=True)


----
Notes & troubleshooting
- If pip fails to install the wheel in your runtime, try restarting the runtime and re-running the install cell.
- On Colab the environment is behind CGNAT / gVisor; settings only help for outbound connections and faster discovery — they cannot force inbound connectivity.
- If you want a non-blocking design, I can convert the engine to async_add_torrent + alert handling.

If you want, I will:
- produce a zipped .ipynb file ready to upload, OR
- add an asynchronous version that posts alerts and returns the handle immediately, OR
- include optional auto‑zipping into Drive after completion.

Would you like any of those? 🚀